In [ ]:
!pip install psycopg2-binary

In [ ]:
import pandas as pd

In [1]:
import psycopg2


DB_CONFIG = {
    "dbname": "cs623_project",
    "user": "postgres",
    "password": "Jumpstart718",  
    "host": "localhost",
    "port": "5432"
}

def get_connection():
    
    return psycopg2.connect(**DB_CONFIG)

print("Database configuration loaded")

Database configuration loaded


In [2]:
def show_all_tables():
    """Display current state of all tables"""
    conn = get_connection()
    cur = conn.cursor()
    
    print("=" * 60)
    
    print("PRODUCT TABLE:")
    print("-" * 40)
    cur.execute("SELECT * FROM Product ORDER BY prodid")
    rows = cur.fetchall()
    if rows:
        print(f"{'prodid':<10} {'pname':<15} {'price':<10}")
        print("-" * 40)
        for row in rows:
            print(f"{row[0]:<10} {row[1]:<15} {row[2]:<10}")
    else:
        print("(empty)")
    
    print("\nDEPOT TABLE:")
    print("-" * 40)
    cur.execute("SELECT * FROM Depot ORDER BY depid")
    rows = cur.fetchall()
    if rows:
        print(f"{'depid':<10} {'addr':<15} {'volume':<10}")
        print("-" * 40)
        for row in rows:
            print(f"{row[0]:<10} {row[1]:<15} {row[2]:<10}")
    else:
        print("(empty)")
    
    print("\nSTOCK TABLE:")
    print("-" * 40)
    cur.execute("SELECT * FROM Stock ORDER BY prodid, depid")
    rows = cur.fetchall()
    if rows:
        print(f"{'prodid':<10} {'depid':<10} {'quantity':<10}")
        print("-" * 40)
        for row in rows:
            print(f"{row[0]:<10} {row[1]:<10} {row[2]:<10}")
    else:
        print("(empty)")
    
    print("=" * 60)
    conn.close()

print("Helper function loaded")

Helper function loaded


In [3]:
show_all_tables()

PRODUCT TABLE:
----------------------------------------
prodid     pname           price     
----------------------------------------
p1         tape            2.50      
p2         tv              250.00    
p3         vcr             80.00     

DEPOT TABLE:
----------------------------------------
depid      addr            volume    
----------------------------------------
d1         New York        9000      
d2         Syracuse        6000      

STOCK TABLE:
----------------------------------------
prodid     depid      quantity  
----------------------------------------
p1         d1         1000      
p1         d2         100       
p2         d1         200       
p3         d1         300       
p3         d2         400       


In [4]:
def reset_database():
    """Reset database to initial state"""
    conn = get_connection()
    try:
        conn.autocommit = False
        cur = conn.cursor()
        
        cur.execute("DELETE FROM Stock")
        cur.execute("DELETE FROM Depot")
        cur.execute("DELETE FROM Product")
        
        cur.execute("INSERT INTO Product VALUES ('p1', 'tape', 2.5)")
        cur.execute("INSERT INTO Product VALUES ('p2', 'tv', 250)")
        cur.execute("INSERT INTO Product VALUES ('p3', 'vcr', 80)")
        
        cur.execute("INSERT INTO Depot VALUES ('d1', 'New York', 9000)")
        cur.execute("INSERT INTO Depot VALUES ('d2', 'Syracuse', 6000)")
        
        cur.execute("INSERT INTO Stock VALUES ('p1', 'd1', 1000)")
        cur.execute("INSERT INTO Stock VALUES ('p1', 'd2', 100)")
        cur.execute("INSERT INTO Stock VALUES ('p2', 'd1', 200)")
        cur.execute("INSERT INTO Stock VALUES ('p3', 'd1', 300)")
        cur.execute("INSERT INTO Stock VALUES ('p3', 'd2', 400)")
        
        conn.commit()
        print("Database reset to initial state")
    except Exception as e:
        conn.rollback()
        print(f"Reset failed: {e}")
    finally:
        conn.close()

print("Reset function loaded")

Reset function loaded


In [ ]:
#Transaction 1#

In [5]:
def delete_product(prodid):
    """
    TRANSACTION 1: Delete a product
    CASCADE automatically removes related Stock entries
    """
    conn = get_connection()
    try:
        # ATOMICITY: Start transaction - nothing saves until commit()
        conn.autocommit = False
        cur = conn.cursor()
        
        # CONSISTENCY: CASCADE handles Stock table automatically
        cur.execute("DELETE FROM Product WHERE prodid = %s", (prodid,))
        
        # DURABILITY: Write permanently to disk
        # ISOLATION: NOW other users see the change
        conn.commit()
        print(f"SUCCESS: Product '{prodid}' deleted")
        print(f"  (CASCADE automatically removed {prodid} from Stock)")
        return True
        
    except Exception as e:
        # ATOMICITY: Undo everything on failure
        conn.rollback()
        print(f"FAILED: {e}")
        return False
        
    finally:
        conn.close()

In [6]:
# Run Transaction 1
print("BEFORE:")
show_all_tables()

print("\n>>> Executing Transaction 1: Delete product p1...\n")
delete_product('p1')

print("\nAFTER:")
show_all_tables()

BEFORE:
PRODUCT TABLE:
----------------------------------------
prodid     pname           price     
----------------------------------------
p1         tape            2.50      
p2         tv              250.00    
p3         vcr             80.00     

DEPOT TABLE:
----------------------------------------
depid      addr            volume    
----------------------------------------
d1         New York        9000      
d2         Syracuse        6000      

STOCK TABLE:
----------------------------------------
prodid     depid      quantity  
----------------------------------------
p1         d1         1000      
p1         d2         100       
p2         d1         200       
p3         d1         300       
p3         d2         400       

>>> Executing Transaction 1: Delete product p1...

SUCCESS: Product 'p1' deleted
  (CASCADE automatically removed p1 from Stock)

AFTER:
PRODUCT TABLE:
----------------------------------------
prodid     pname           price     
-------

In [8]:
# Reset for next transaction
reset_database()

Database reset to initial state


In [10]:
#Transaction 2

In [11]:
def delete_depot(depid):
    """
    TRANSACTION 2: Delete a depot
    CASCADE automatically removes related Stock entries
    """
    conn = get_connection()
    try:
        # ATOMICITY: Start transaction
        conn.autocommit = False
        cur = conn.cursor()
        
        # CONSISTENCY: CASCADE handles Stock automatically
        cur.execute("DELETE FROM Depot WHERE depid = %s", (depid,))
        
        # DURABILITY & ISOLATION
        conn.commit()
        print(f"SUCCESS: Depot '{depid}' deleted")
        print(f"  (CASCADE automatically removed {depid} from Stock)")
        return True
        
    except Exception as e:
        # ATOMICITY: Rollback on error
        conn.rollback()
        print(f"FAILED: {e}")
        return False
        
    finally:
        conn.close()

In [12]:
# Run Transaction 2
print("BEFORE:")
show_all_tables()

print("\n>>> Executing Transaction 2: Delete depot d1...\n")
delete_depot('d1')

print("\nAFTER:")
show_all_tables()

BEFORE:
PRODUCT TABLE:
----------------------------------------
prodid     pname           price     
----------------------------------------
p1         tape            2.50      
p2         tv              250.00    
p3         vcr             80.00     

DEPOT TABLE:
----------------------------------------
depid      addr            volume    
----------------------------------------
d1         New York        9000      
d2         Syracuse        6000      

STOCK TABLE:
----------------------------------------
prodid     depid      quantity  
----------------------------------------
p1         d1         1000      
p1         d2         100       
p2         d1         200       
p3         d1         300       
p3         d2         400       

>>> Executing Transaction 2: Delete depot d1...

SUCCESS: Depot 'd1' deleted
  (CASCADE automatically removed d1 from Stock)

AFTER:
PRODUCT TABLE:
----------------------------------------
prodid     pname           price     
-----------

In [13]:
# Reset for next transaction
reset_database()

Database reset to initial state


In [14]:
#Transaction 3

In [15]:
def update_product_name(old_prodid, new_prodid):
    """
    TRANSACTION 3: Rename a product
    ON UPDATE CASCADE automatically updates Stock references
    """
    conn = get_connection()
    try:
        # ATOMICITY: Begin transaction
        conn.autocommit = False
        cur = conn.cursor()
        
        # CONSISTENCY: ON UPDATE CASCADE propagates to Stock
        cur.execute("UPDATE Product SET prodid = %s WHERE prodid = %s", 
                    (new_prodid, old_prodid))
        
        # DURABILITY: Save permanently
        conn.commit()
        print(f"SUCCESS: Product '{old_prodid}' renamed to '{new_prodid}'")
        print(f"  (CASCADE automatically updated Stock references)")
        return True
        
    except Exception as e:
        # ATOMICITY: Rollback on error
        conn.rollback()
        print(f"FAILED: {e}")
        return False
        
    finally:
        conn.close()

In [16]:
# Run Transaction 3
print("BEFORE:")
show_all_tables()

print("\n>>> Executing Transaction 3: Rename p1 to pp1...\n")
update_product_name('p1', 'pp1')

print("\nAFTER:")
show_all_tables()

BEFORE:
PRODUCT TABLE:
----------------------------------------
prodid     pname           price     
----------------------------------------
p1         tape            2.50      
p2         tv              250.00    
p3         vcr             80.00     

DEPOT TABLE:
----------------------------------------
depid      addr            volume    
----------------------------------------
d1         New York        9000      
d2         Syracuse        6000      

STOCK TABLE:
----------------------------------------
prodid     depid      quantity  
----------------------------------------
p1         d1         1000      
p1         d2         100       
p2         d1         200       
p3         d1         300       
p3         d2         400       

>>> Executing Transaction 3: Rename p1 to pp1...

SUCCESS: Product 'p1' renamed to 'pp1'
  (CASCADE automatically updated Stock references)

AFTER:
PRODUCT TABLE:
----------------------------------------
prodid     pname           price  

In [17]:
# Reset for next transaction
reset_database()

Database reset to initial state


In [ ]:
#Transaction 4

In [18]:
def update_depot_name(old_depid, new_depid):
    """
    TRANSACTION 4: Rename a depot
    ON UPDATE CASCADE automatically updates Stock references
    """
    conn = get_connection()
    try:
        # ATOMICITY: Begin transaction
        conn.autocommit = False
        cur = conn.cursor()
        
        # CONSISTENCY: ON UPDATE CASCADE propagates to Stock
        cur.execute("UPDATE Depot SET depid = %s WHERE depid = %s", 
                    (new_depid, old_depid))
        
        # DURABILITY: Save permanently
        conn.commit()
        print(f"SUCCESS: Depot '{old_depid}' renamed to '{new_depid}'")
        print(f"  (CASCADE automatically updated Stock references)")
        return True
        
    except Exception as e:
        # ATOMICITY: Rollback on error
        conn.rollback()
        print(f"FAILED: {e}")
        return False
        
    finally:
        conn.close()

In [19]:
# Run Transaction 4
print("BEFORE:")
show_all_tables()

print("\n>>> Executing Transaction 4: Rename d1 to dd1...\n")
update_depot_name('d1', 'dd1')

print("\nAFTER:")
show_all_tables()

BEFORE:
PRODUCT TABLE:
----------------------------------------
prodid     pname           price     
----------------------------------------
p1         tape            2.50      
p2         tv              250.00    
p3         vcr             80.00     

DEPOT TABLE:
----------------------------------------
depid      addr            volume    
----------------------------------------
d1         New York        9000      
d2         Syracuse        6000      

STOCK TABLE:
----------------------------------------
prodid     depid      quantity  
----------------------------------------
p1         d1         1000      
p1         d2         100       
p2         d1         200       
p3         d1         300       
p3         d2         400       

>>> Executing Transaction 4: Rename d1 to dd1...

SUCCESS: Depot 'd1' renamed to 'dd1'
  (CASCADE automatically updated Stock references)

AFTER:
PRODUCT TABLE:
----------------------------------------
prodid     pname           price    

In [20]:
# Reset for next transaction
reset_database()

Database reset to initial state


In [ ]:
#Trasnsaction 5

In [21]:
def add_product_and_stock(prodid, pname, price, depid, quantity):
    """
    TRANSACTION 5: Add product AND stock entry atomically
    Both inserts succeed together or both fail together
    """
    conn = get_connection()
    try:
        # ATOMICITY: Groups BOTH inserts in ONE transaction
        conn.autocommit = False
        cur = conn.cursor()
        
        # CONSISTENCY: Must insert Product FIRST (Stock references it)
        cur.execute("INSERT INTO Product (prodid, pname, price) VALUES (%s, %s, %s)", 
                    (prodid, pname, price))
        print(f"  -> Inserted into Product (not committed yet)")
        
        # Second insert - if this fails, Product insert is rolled back too
        cur.execute("INSERT INTO Stock (prodid, depid, quantity) VALUES (%s, %s, %s)", 
                    (prodid, depid, quantity))
        print(f"  -> Inserted into Stock (not committed yet)")
        
        # DURABILITY: NOW both are saved permanently
        # ISOLATION: NOW other users can see both records
        conn.commit()
        print(f"SUCCESS: Added product '{prodid}' and stock entry")
        print(f"  (Both inserts committed together - ATOMICITY)")
        return True
        
    except Exception as e:
        # ATOMICITY: BOTH inserts are undone
        conn.rollback()
        print(f"FAILED: {e}")
        print(f"  (rollback() undid BOTH inserts)")
        return False
        
    finally:
        conn.close()

In [22]:
# Run Transaction 5: Add product p100 (cd, $5) with stock in d2 (quantity 50)
print("BEFORE:")
show_all_tables()

print("\n>>> Executing Transaction 5: Add product p100 and stock entry...\n")
add_product_and_stock('p100', 'cd', 5, 'd2', 50)

print("\nAFTER:")
show_all_tables()

BEFORE:
PRODUCT TABLE:
----------------------------------------
prodid     pname           price     
----------------------------------------
p1         tape            2.50      
p2         tv              250.00    
p3         vcr             80.00     

DEPOT TABLE:
----------------------------------------
depid      addr            volume    
----------------------------------------
d1         New York        9000      
d2         Syracuse        6000      

STOCK TABLE:
----------------------------------------
prodid     depid      quantity  
----------------------------------------
p1         d1         1000      
p1         d2         100       
p2         d1         200       
p3         d1         300       
p3         d2         400       

>>> Executing Transaction 5: Add product p100 and stock entry...

  -> Inserted into Product (not committed yet)
  -> Inserted into Stock (not committed yet)
SUCCESS: Added product 'p100' and stock entry
  (Both inserts committed together

In [23]:
# Reset for next transaction
reset_database()

Database reset to initial state


In [24]:
#Transaction 6

In [25]:
def add_depot_and_stock(depid, addr, volume, prodid, quantity):
    """
    TRANSACTION 6: Add depot AND stock entry atomically
    Both inserts succeed together or both fail together
    """
    conn = get_connection()
    try:
        # ATOMICITY: Groups BOTH inserts in ONE transaction
        conn.autocommit = False
        cur = conn.cursor()
        
        # CONSISTENCY: Insert Depot first (Stock references it)
        cur.execute("INSERT INTO Depot (depid, addr, volume) VALUES (%s, %s, %s)", 
                    (depid, addr, volume))
        print(f"  -> Inserted into Depot (not committed yet)")
        
        # Stock insert - prodid must exist in Product
        cur.execute("INSERT INTO Stock (prodid, depid, quantity) VALUES (%s, %s, %s)", 
                    (prodid, depid, quantity))
        print(f"  -> Inserted into Stock (not committed yet)")
        
        # DURABILITY & ISOLATION
        conn.commit()
        print(f"SUCCESS: Added depot '{depid}' and stock entry")
        print(f"  (Both inserts committed together - ATOMICITY)")
        return True
        
    except Exception as e:
        # ATOMICITY: BOTH inserts are undone
        conn.rollback()
        print(f"FAILED: {e}")
        print(f"  (rollback() undid BOTH inserts)")
        return False
        
    finally:
        conn.close()

In [26]:
# Run Transaction 6: Add depot d100 (Chicago, volume 100) with stock for p1 (quantity 100)
print("BEFORE:")
show_all_tables()

print("\n>>> Executing Transaction 6: Add depot d100 and stock entry...\n")
add_depot_and_stock('d100', 'Chicago', 100, 'p1', 100)

print("\nAFTER:")
show_all_tables()

BEFORE:
PRODUCT TABLE:
----------------------------------------
prodid     pname           price     
----------------------------------------
p1         tape            2.50      
p2         tv              250.00    
p3         vcr             80.00     

DEPOT TABLE:
----------------------------------------
depid      addr            volume    
----------------------------------------
d1         New York        9000      
d2         Syracuse        6000      

STOCK TABLE:
----------------------------------------
prodid     depid      quantity  
----------------------------------------
p1         d1         1000      
p1         d2         100       
p2         d1         200       
p3         d1         300       
p3         d2         400       

>>> Executing Transaction 6: Add depot d100 and stock entry...

  -> Inserted into Depot (not committed yet)
  -> Inserted into Stock (not committed yet)
SUCCESS: Added depot 'd100' and stock entry
  (Both inserts committed together - ATO

In [ ]:
reset_database()